In [ ]:
import os
import numpy as np
import glob
import cv2

import matplotlib.pyplot as plt
import seaborn as sns

import sys
sys.path.append('.../bats-code')
import bat_functions as bf

In [ ]:
root_folder = ".../kasanka-bats/processed/deep-learning"
observations_root = os.path.join(root_folder, "observations")
all_observations = {}
day_folders = sorted(glob.glob(os.path.join(observations_root, '*')))
for day_folder in day_folders:
    obs_files = sorted(glob.glob(os.path.join(day_folder, '*.npy')))
    date = os.path.basename(day_folder)
    all_observations[date] = {}
    for obs_file in obs_files:
        camera = os.path.splitext(obs_file)[0].split('-')[-1]
        obs = np.load(obs_file, allow_pickle=True)
        # .item() to get dict from inside the array that was wrapped around
        # it when using np.save()
        all_observations[date][camera] = obs.item()
        
# Remove observations to exclude (because camera ran out of batteries etc.)
exclude=True
# Manually exclude cameras that had issues
all_observations['17Nov']['MusoleParking']['exclude'] = True
all_observations['18Nov']['MusolePath']['exclude'] = True
all_observations['20Nov']['MusolePath']['exclude'] = True
if exclude:
    good_obs = {}
    for date, day_obs in all_observations.items():
        good_obs[date] = {}
        for camera, obs in day_obs.items():
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    continue
            good_obs[date][camera] = obs
    all_observations = good_obs

In [ ]:
plots_save_folder = '.../bats-data/plots/observation-summary'
os.makedirs(plots_save_folder, exist_ok=True)

In [ ]:
shift = 0 # loss on each side from not padding during detection (48)
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 
HCONST = 1454.9 # pixels

In [ ]:
obs.keys()

### Bat height

In [ ]:
for camera, obs in day_obs.items():
    print(camera)

In [ ]:
camera_names = {'16Nov':['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '17Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking2', 'FibwePublic',
                         'MusoleTower', 'MusolePath2', 'MusoleParking',
                         'Sunset', 'Puku'],
                '18Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
                '19Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusolePath', 'MusoleParking',
                         'Sunset', 'Puku'],
                '20Nov': ['NotChyniangale', 'Chyniangale',
                         'BBC', 'FibweParking', 'FibwePublic',
                         'MusoleTower', 'MusoleParking',
                         'Sunset', 'Puku'],
               }

In [ ]:
# all wing measures


save_fig = False


min_track_length = 100

dates = ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']
# dates = ['16Nov']    
wing_validation_file = '.../bats-data/wing-validation/combined_wing_validation_info.csv'
wing_correction_info = bf.get_wing_correction_distributions(
    wing_validation_file, num_darkness_bins=4, kde_bw_scale=.25, should_plot=False
)
wing_correction_kdes, darkness_bins = wing_correction_info

mean_height = {}

for date, day_obs in all_observations.items():
    
    
    all_cameras = []
    all_heights = []
    
    mean_height[date] = {}
    
#     for camera, obs in day_obs.items():
    for camera in camera_names[date]:
        obs = day_obs[camera] 
        
        correction_scale, kde_inds = bf.get_kde_samples(
            obs, wing_correction_kdes, darkness_bins)
        
        all_wings = []
        short_wings = []
        frame_nums = []
        cameras = []
        correction = []
        
        for wing, track_length, frame, scale in zip(obs['mean_wing'], obs['track_length'], obs['frames'], correction_scale):
            if track_length >= min_track_length:
                all_wings.append(wing)
                frame_nums.append(frame)
                cameras.append(camera)
                correction.append(scale)
            else:
                short_wings.append(wing)
                
        

        all_wings = np.array(all_wings)
        correction_scale = np.array(correction)
        biased_wing = bf.correct_wingspan(all_wings, 
                                          correction_scale)
        biased_wing = np.maximum(biased_wing, 2) # No wingspans smaller than 2 pixels
                
        heights = bf.calculate_height(biased_wing, HCONST, WINGSPAN)
#         short_heights = bf.calculate_height(np.array(short_wings), HCONST, WINGSPAN)

#         plt.figure()
#         _ = plt.hist([heights, short_heights], bins=100, stacked=True)

#         plt.title(camera + ' all heights')
#         sorted_framenum = [fn for _, fn in sorted(zip(heights, frame_nums))]
        sorted_heights = sorted(heights)
#         short_sorted_heights = sorted(short_heights)

        percent = 0.999
#         plt.figure()
#         all_heights = [sorted_heights[:int(len(sorted_heights)*percent)],
#                        short_sorted_heights[:int(len(short_sorted_heights)*percent)]]
#         _ = plt.hist(sorted_heights[:int(len(sorted_heights)*percent)], bins=100)
#         plt.title(camera+' 1.0 heights')
        
#         plt.figure()
#         plt.scatter(np.abs(np.array(frame_nums)), heights, alpha=0.05)
        
        heights = sorted_heights[:int(len(sorted_heights)*percent)]
        
        mean_height[date][camera] = np.mean(heights)
        
        cameras = cameras[:int(len(sorted_heights)*percent)]
        all_heights.extend(heights)
        all_cameras.extend(cameras)
        
#             plt.figure()
#             _ = plt.hist(all_wings, bins=100)

#             plt.figure()
#             _ = plt.hist([all_wings, short_wings], stacked=True, bins=100)

#         sorted_wings = sorted(all_wings)
#         short_sorted_wings = sorted(short_wings)
#         percent = .999
#         print(len(all_wings) * percent, len(all_wings)*(1-percent))
#         plt.figure()
#         _ = plt.hist([sorted_wings[:int(len(sorted_wings)*percent)], short_sorted_wings[:int(len(short_sorted_wings)*percent)]], bins=100, stacked=True)
#         plt.title(camera + ' all wing lengths')   
    plt.figure()
    sns.violinplot(x=all_cameras, y=all_heights)
    plt.xticks(rotation=90)
    
    plt.ylabel('Height (meters)')
    
    title = f'Corrected height distribution {date} {camera} {percent} min track length {min_track_length}'
    
    if save_fig:
        bf.save_fig(plots_save_folder, title)
    

                

# heights = bf.calculate_height(np.array(all_wings), HCONST, WINGSPAN)
# short_heights = bf.calculate_height(np.array(short_wings), HCONST, WINGSPAN)

# plt.figure()
# _ = plt.hist([heights, short_heights], bins=100, stacked=True)

# sorted_heights = sorted(heights)

# percent = .999
# plt.figure()
# _ = plt.hist(sorted_heights[:int(len(sorted_heights)*percent)], bins=100)
                
# plt.figure()
# _ = plt.hist(all_wings, bins=100)

# plt.figure()
# _ = plt.hist([all_wings, short_wings], stacked=True, bins=100)

# sorted_wings = sorted(all_wings)
# short_sorted_wings = sorted(short_wings)
# percent = .999
# print(len(all_wings) * percent, len(all_wings)*(1-percent))
# plt.figure()
# _ = plt.hist([sorted_wings[:int(len(sorted_wings)*percent)], short_sorted_wings[:int(len(short_sorted_wings)*percent)]], bins=100, stacked=True)


In [ ]:
shift = 0 # loss on each side from not padding during detection (48)
FRAME_WIDTH = 2704 - (2 * shift)
WINGSPAN = .8 # meters, max extent while flying 

center_utm = {'middle': np.array([200450, 8606950]),
              'right': np.array([200800, 8606900])}

camera_locations = {'FibweParking2': [-12.5903393, 30.2525047],	
                    'FibweParking': [-12.5903393, 30.2525047],
                    'Chyniangale': [-12.5851284, 30.245529],	
                    'BBC': [-12.5863538, 30.2484985],
                    'Sunset': [-12.585784, 30.240003],
                    'NotChyniangale': [-12.5849206,	30.2436135],
                    'MusoleParking': [-12.58787, 30.2401],	
                    'MusolePath2': [-12.589544,	30.242488],	
                    'MusolePath': [-12.589544,	30.242488],
                    'Puku': [-12.584838, 30.24137],	
                    'FibwePublic': [-12.592537, 30.2515924],	
                    'MusoleTower': [-12.589434, 30.244736],
                    }
all_camera_utms = bf.latlong_dict_to_utm(camera_locations)

In [ ]:
axis_labels_day_ind = 0
should_save=False

center = 'middle'

save_folder = '.../bats-data/plots/center-summary-figure'
os.makedirs(save_folder, exist_ok=True)

all_figs = []
all_axs = []

for day_ind, (date, day) in enumerate(all_observations.items()):
    fig, axs = plt.subplots(1, 3, figsize=(20, 6), 
                        subplot_kw=dict(polar=True))
    all_figs.append(fig)
    all_axs.append(axs)

    
max_bats_per_degree = 0
max_bats_per_camera = 0
max_height = 0

for day_ind, ((date, day), fig, axs) in enumerate(zip(all_observations.items(), all_figs, all_axs)):


#     fig, axs = plt.subplots(1, 3, figsize=(20, 6), 
#                         subplot_kw=dict(polar=True))

    day_total, day_total_mean = bf.get_day_total(day, center_utm[center], all_camera_utms, 
                                              FRAME_WIDTH, WINGSPAN, exclude=exclude)
    fractions = []
    cameras = []
    totals = []
    contribution = []
    angles = []
    heights = []
    
    camera_utms = bf.get_camera_locations(day, all_camera_utms, exclude=True)
    
    print(date, len(camera_utms))

    camera_angles = bf.get_camera_angles(camera_utms, center_utm[center])
    for camera, obs in day.items():
        if exclude:
            if 'exclude' in obs.keys():
                if obs['exclude']:
                    print('excluding observation...')
                    continue
        fractions.append(obs['fraction_total'])
        cameras.append(camera)
        totals.append(obs['total']/360)
        contribution.append(obs['total'] * obs['fraction_total'])
        angles.append(camera_angles[camera])
        heights.append(mean_height[date][camera])

    max_bats_per_degree_day = np.max(totals)
    if max_bats_per_degree_day > max_bats_per_degree:
        max_bats_per_degree = max_bats_per_degree_day
    max_bats_per_camera_day = np.max(contribution)
    if max_bats_per_camera_day > max_bats_per_camera:
        max_bats_per_camera = max_bats_per_camera_day
    max_height_day = np.max(height)
    if max_height_day > max_height:
        max_height = max_height_day

    # fractions = [f for _,f in sorted(zip(angles,fractions))]
#     frac_ind = 3
#     axs[frac_ind].set_thetalim(-np.pi, np.pi)
#     axs[frac_ind].scatter(angles, fractions, s=30)
# #     if axis_labels_day_ind == day_ind:
#     axs[frac_ind].set_xticks(angles)
#     _ = axs[frac_ind].set_xticklabels(cameras)
#     axs[frac_ind].set_title('Fraction', size='x-large')

    total_ind = 1
    axs[total_ind].set_thetalim(-np.pi, np.pi)
    axs[total_ind].scatter(angles, totals)
#     if axis_labels_day_ind == day_ind:
    axs[total_ind].set_xticks(angles)
    _ = axs[total_ind].set_xticklabels(cameras)
    axs[total_ind].set_title('Bats per degree', size='x-large')

    contrib_ind = 0
    axs[contrib_ind].set_thetalim(-np.pi, np.pi)
#     axs[contrib_ind].scatter(angles, contribution, s=30)
    axs[contrib_ind].bar(angles, contribution)
#     if axis_labels_day_ind == day_ind:
    axs[contrib_ind].set_xticks(angles)
    _ = axs[contrib_ind].set_xticklabels(cameras)
    axs[contrib_ind].set_title('Bats from camera', size='x-large')
    
    height_ind = 2
    axs[height_ind].set_thetalim(-np.pi, np.pi)
    axs[height_ind].scatter(angles, heights, s=30)
#     if axis_labels_day_ind == day_ind:
    axs[height_ind].set_xticks(angles)
    _ = axs[height_ind].set_xticklabels(cameras)
    axs[height_ind].set_title('Bats average height (meters)', size='x-large')

    pad = 60
#         left_label = f'{date} \n {int(np.sum(contribution)):,} bats'
#         axs[day_ind,0].annotate(left_label, xy=(0, 0.5), 
#                                 xytext=(-axs[day_ind,0].yaxis.labelpad - pad, 0),
#                                 xycoords=axs[day_ind,0].yaxis.label, 
#                                 textcoords='offset points',
#                                 size='xx-large', ha='right', va='center'
#                                )
#     for day_ind, (date, day) in enumerate(all_observations.items()):
for fig, axs in zip(all_figs, all_axs):
    axs[contrib_ind].set_rlim(0, max_bats_per_camera * 1.1)
    axs[total_ind].set_rlim(0, max_bats_per_degree * 1.1)
    axs[height_ind].set_rlim(0, max_height * 1.1)

title = f'summary-for-days-combined-center-{center}'
fig.suptitle(title.replace('-', ' '), size=30, va='center')
if should_save:
    fig.savefig(os.path.join(save_folder, title + '.png'), bbox_inches='tight')

In [ ]:
all_heights[:10]

In [ ]:
dates = ['16Nov', '17Nov', '18Nov', '19Nov', '20Nov']
# dates = ['16Nov']    
wing_validation_file = '.../bats-data/wing-validation/combined_wing_validation_info.csv'
wing_correction_info = bf.get_wing_correction_distributions(
    wing_validation_file, num_darkness_bins=4, kde_bw_scale=.25, should_plot=False
)
wing_correction_kdes, darkness_bins = wing_correction_info

for date, day_obs in all_observations.items():
    for camera, obs in day_obs.items():
        
        assert len(obs['darkness']) == len(obs['mean_wing'])
        test_inds = []
        test_heights = []
        for i in range(10):

            correction_scale, kde_inds = bf.get_kde_samples(
                obs, wing_correction_kdes, darkness_bins)


            biased_wing = bf.correct_wingspan(obs['mean_wing'], 
                                              correction_scale)
            biased_wing = np.maximum(biased_wing, 2) # No wingspans smaller than 2 pixels
            heights = bf.calculate_height(
                biased_wing, HCONST, WINGSPAN)
            test_heights.extend(heights)
            test_inds.extend([i for _ in heights])
#             plt.figure()
#             _= plt.hist(heights, bins=100)
        plt.figure()
        sns.violinplot(x=test_inds, y=test_heights)
    break


In [ ]:
np.ones(wing_samples)*measured_wing - correction_scale

In [ ]:
correction_scale.shape